# Tutorial 4: Análisis de Itemsets frecuentes y Reglas de Asociación

*Mining transactional data*. En este tutorial veremos cómo analizar los elementos frecuentes y cómo construir las reglas de asociación, basados en medidas de interés.

## Conceptos básicos

En el análisis de reglas de asociación (Association Rules) y de conjuntos de objetos frecuentes (Frequent Itemset Analysis) tenemos varios conceptos que son importantes de recordar para entender correctamente lo que hacen las reglas de asociación. Tomemos como ejemplo los datos de compras en un supermercado.

- **Item**: Un objeto. Por ejemplo: leche, pañales, cerveza.
- **Itemset**: Un conjunto de uno o más objetos. Por ejemplo: {pan, bebida}, {pan, leche, cerveza}.
- **Transacción**: Una fila del dataset. Una transacción también es un itemset, ya que corresponde a un conjunto de objetos, pero una transacción es un dato del que disponemos y no un itemset arbitrario. Por ejemplo, una compra en el supermercado es una transacción, y esta puede contener múltiples objetos: {leche, pañales}, mientras que otra puede ser: {pan, leche, huevos}.
- **Dataset**: Conjunto de transacciones. Corresponde a las compras del supermercado de las que disponemos.
- **Regla de asociación**: Una regla del estilo $X \rightarrow Y$, donde $X$ e $Y$ son itemsets, y $X\cap Y =\emptyset$. Por ejemplo, {leche, yogurt} $\rightarrow$ {pan}. El lado izquierdo de la regla (LHS) se conoce como antecedente y el lado derecho (RHS) como consecuente. 

Ojo que una regla de asociación *no* es una implicación lógica. Es decir, no necesariamente existe una relación de causalidad entre $X$ e $Y$, sino que de co-ocurrencia.

Existen distintas medidas de interés sobre itemsets y reglas. Entre las más importantes están *support*, *confidence*, y *lift*:

$$\sigma(X) = \text{# de veces que aparece }X \text{ en el dataset}$$
$$\text{support}(X) = \frac{\sigma(X)}{N}$$

$$\text{support}(X \rightarrow Y) = \frac{\sigma(X \cup Y)}{N}$$

$$\text{confidence}(X \rightarrow Y) = \frac{\text{support}(X\rightarrow Y)}{\text{support}(X)} = \frac{\sigma(X \cup Y)}{\sigma(X)}$$
$$\text{lift}(X\rightarrow Y) = \frac{\text{confidence}(X\rightarrow Y)}{\text{support}(Y)}$$

Donde $N$ es la cantidad de transacciones (el tamaño del dataset).

## Preámbulo

Usaremos la librería `mlxtend`, que contiene algunas herramientas adicionales que no se encuentran en `sklearn`.

In [ ]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Primero creamos un dataset de transacciones. En este caso, cada transacción corresponde a las compras hechas en un almacén.

In [ ]:
dataset = [['Leche', 'Cebollas', 'Pisco', 'Porotos', 'Huevos', 'Yogurt'],
           ['Eneldo', 'Cebollas', 'Pisco', 'Porotos', 'Huevos', 'Yogurt'],
           ['Leche', 'Manzanas', 'Porotos', 'Huevos'],
           ['Leche', 'Tomillo', 'Espinaca', 'Porotos', 'Yogurt'],
           ['Espinaca', 'Cebollas', 'Cebollas', 'Porotos', 'Helado', 'Huevos']]

Importamos el `TransactionEncoder` que convierte los datos en una matriz de ocurrencias: cada columna corresponde a alguno de los items, cada fila es la misma transacción, y cada celda es 0 o 1 dependiendo de si la transacción contiene al item.

¿A qué se parece este encoder? ¿En qué se diferencia?

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Cebollas,Eneldo,Espinaca,Helado,Huevos,Leche,Manzanas,Pisco,Porotos,Tomillo,Yogurt
0,True,False,False,False,True,True,False,True,True,False,True
1,True,True,False,False,True,False,False,True,True,False,True
2,False,False,False,False,True,True,True,False,True,False,False
3,False,False,True,False,False,True,False,False,True,True,True
4,True,False,True,True,True,False,False,False,True,False,False


## Itemsets frecuentes con Apriori

Para observar los itemsets más frecuentes se definen umbrales (thresholds) de `soporte` y se generan reglas cuya medida de interés sea mayor o igual a cada uno de estos umbrales.

Usamos el método `apriori` para generar itemsets frecuentes con un soporte mínimo. Con el parámetro `use_colnames` podemos recuperar los nombres de las columnas en cada itemset:

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.6,(Cebollas)
1,0.8,(Huevos)
2,0.6,(Leche)
3,1.0,(Porotos)
4,0.6,(Yogurt)
5,0.6,"(Cebollas, Huevos)"
6,0.6,"(Cebollas, Porotos)"
7,0.8,"(Huevos, Porotos)"
8,0.6,"(Leche, Porotos)"
9,0.6,"(Yogurt, Porotos)"


El output de `apriori` es un DataFrame de pandas, por lo que podemos manipularlo igual que a un DataFrame cualquiera.

Por ejemplo, vamos a ordenarlo por soporte y a agregar una nueva columna que indique el tamaño del itemset:

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)

frequent_itemsets

,support,itemsets,length
3,1.0,(Porotos),1
1,0.8,(Huevos),1
7,0.8,"(Huevos, Porotos)",2
0,0.6,(Cebollas),1
2,0.6,(Leche),1
4,0.6,(Yogurt),1
5,0.6,"(Cebollas, Huevos)",2
6,0.6,"(Cebollas, Porotos)",2
8,0.6,"(Leche, Porotos)",2
9,0.6,"(Yogurt, Porotos)",2


Podemos hacer filtros más elaborados usando pandas. 

Por ejemplo: ¿cuáles itemsets de largo 2 tienen soporte mayor o igual a 80%?

In [ ]:
frequent_itemsets.loc[(frequent_itemsets['length'] == 2) &
                      (frequent_itemsets['support'] >= 0.8)]

,support,itemsets,length
7,0.8,"(Huevos, Porotos)",2


¿Cuáles itemsets corresponden a huevos y cebollas?

In [ ]:
#frequent_itemsets.loc[ frequent_itemsets['itemsets'] == {'Cebollas', 'Huevos'} ]
## {a, b, c, ...} es un conjunto (frozenset), por lo que el orden no importa:

frequent_itemsets.loc[ frequent_itemsets['itemsets'] == {'Huevos', 'Cebollas'} ]

,support,itemsets,length
5,0.6,"(Cebollas, Huevos)",2


In [ ]:
type(frequent_itemsets['itemsets'].iloc[0])

frozenset

In [ ]:
## y para consultar cuáles itemsets contienen al menos ciertos items:
frequent_itemsets.loc[
                      frequent_itemsets['itemsets'].apply(lambda conjunto: conjunto.issuperset({'Huevos', 'Cebollas'}))
                      ]

,support,itemsets,length
5,0.6,"(Cebollas, Huevos)",2
10,0.6,"(Cebollas, Huevos, Porotos)",3


## Reglas de Asociación

La función `association_rules` permite generar reglas usando un umbral de support, confidence, u otras medidas de interés:

In [ ]:
from mlxtend.frequent_patterns import association_rules
association_rules?

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Huevos),(Porotos),0.8,1.0,0.8,1.00,1.00,0.00,inf
1,(Porotos),(Huevos),1.0,0.8,0.8,0.80,1.00,0.00,1.0
2,(Cebollas),(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf
3,(Huevos),(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6
4,(Cebollas),(Porotos),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(Leche),(Porotos),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,(Yogurt),(Porotos),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,"(Cebollas, Huevos)",(Porotos),0.6,1.0,0.6,1.00,1.00,0.00,inf
8,"(Cebollas, Porotos)",(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf
9,"(Huevos, Porotos)",(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6


El `antecedents` o `LHS` ("left hand side") es el lado izquierdo de la regla. `consequents` o `RHS` es el lado derecho.

Por ejemplo, la primera regla es $\{\text{Porotos}\} \rightarrow \{\text{Huevos}\}$

Podemos cambiar la medida de interés y aplicar otro filtro para la generación de reglas. Si queremos aplicar más de un filtro, podemos hacerlo posterior a la generación de las reglas.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cebollas),(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf
1,(Huevos),(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6
2,"(Cebollas, Porotos)",(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf
3,"(Huevos, Porotos)",(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6
4,(Cebollas),"(Huevos, Porotos)",0.6,0.8,0.6,1.00,1.25,0.12,inf
5,(Huevos),"(Cebollas, Porotos)",0.8,0.6,0.6,0.75,1.25,0.12,1.6


Para filtrar por el tamaño del LHS, del RHS, o ambos, podemos crear una columna nueva con el tamaño del conjunto y luego hacer el filtro correspondiente

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Cebollas),(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf,1
1,(Huevos),(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1
2,"(Cebollas, Porotos)",(Huevos),0.6,0.8,0.6,1.00,1.25,0.12,inf,2
3,"(Huevos, Porotos)",(Cebollas),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2
4,(Cebollas),"(Huevos, Porotos)",0.6,0.8,0.6,1.00,1.25,0.12,inf,1
5,(Huevos),"(Cebollas, Porotos)",0.8,0.6,0.6,0.75,1.25,0.12,1.6,1


In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2,"(Cebollas, Porotos)",(Huevos),0.6,0.8,0.6,1.0,1.25,0.12,inf,2


In [ ]:
rules[rules['antecedents'] == {'Porotos', 'Cebollas'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2,"(Cebollas, Porotos)",(Huevos),0.6,0.8,0.6,1.0,1.25,0.12,inf,2


Llegados a este punto, es posible que se quiera ver cuánta oportunidad hay de utilizar la popularidad de un producto para impulsar las ventas de otro. 

## Referencias

*   http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
*   https://pbpython.com/market-basket-analysis.html#:~:text=Association%20rules%20are%20normally%20written,%7BBeer%7D%20is%20the%20consequent.
*  Usando la librería PyCaret: https://github.com/pycaret/pycaret/blob/master/tutorials/Association%20Rule%20Mining%20Tutorial%20-%20ARUL01.ipynb